# Ques 2: To get the top seller we did the following things

* At first we created a dictionary where as key we stored the agent id and value for 0 (later to be filled by sale)
* Then we merge each transaction csv with the productcsv based on 'product'
* After that we group_by that newly created csv by 'agent' and get sum for the prices
* Then we added the prices according to the agent key in dictionary from the newly created group_by dataframe.
* Finally we sort the dictionary in descending order based on the prices to get top seller agent

In [2]:
import pandas as pd

# Please change the path accordingly
agent = pd.read_csv('../input/synaptican-dataset/agent.csv')
product = pd.read_csv('../input/synaptican-dataset/product.csv')


# Merging the product and price with transaction base on unique product

In [3]:
df = pd.merge(transaction, product[['product', 'price']], on=['product'])

In [4]:
agent_sale_dict = {}

for each_agent in agent['agent']:
    agent_sale_dict[each_agent] = 0

# Loading all Transaction Data

In [5]:
import tqdm
#agent_sale_dict = {}
for i in range(20):
    if i<10:
        i = f'0{i}'
    
    #Change the path
    transaction = pd.read_csv(f'../input/synaptican-dataset/transaction/part-000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv')
    df = pd.merge(transaction, product[['product', 'price']], on=['product'])
    df = df.groupby(['agent']).sum().reset_index()

    for i in tqdm.tqdm(range(len(df)), desc=f"Doint for {i} csv:"):
        agent_sale_dict[df.iloc[i]['agent']] += df.iloc[i]['price']

In [6]:
agent_sale_dict_sorted = dict(sorted(agent_sale_dict.items(), key=lambda item: item[1], reverse=True))

### These are the top seller agent and their sales

In [7]:
i=0
for k, v in agent_sale_dict_sorted.items():
    if i==100:
        break
    print(k)
    print(v)
    i+=1
    

# Getting isolated high seller agent who doesn't have any seller agent nearby within 500 meter

* First we created a dictionary where key is the seller ID and value is a initialized with dummy list where we have (lat, lon, timestapmp) will be later filled
* Then we iterate through the agent location and updated the dictionary based on the agent's latest location (the later dated location means the latest location)
* The agent who doesn't have any location data is replaced by (0,0,datetime.datetime(1980,6,1)) data , basically on the equator
* Then from the dictionary we created a agent location dataframe and sorted the dataframe based on latitude and longitude
* Then we plot the agent location on the map and also the top selling agent in seperate map for visualization
* Then we created a top_seller dataframe for the top 100 seller agent
* We iterate through those agent and find their nearby agent and calculate the distace based on location
* Those agent who did not have any agent nearby was appended to a list called 'final_list'
* So we find out among the top 100 seller there are 40 agents who did not have any agent nearby 500 meter

In [8]:
import datetime
import tqdm
from dateutil import parser
import math
agent_loc_ts = {}

for single_agent in tqdm.tqdm(agent['agent'].unique()):
    agent_loc_ts[single_agent] = [0,0,datetime.datetime(1980,6,1)]

In [9]:
for i in tqdm.tqdm(range(len(agent)), desc="Generating agent latest loc:"):
    date = parser.parse(agent.iloc[i]['ts'])
    if agent_loc_ts[agent.iloc[i]['agent']][-1] < date and not math.isnan(agent.iloc[i]['lat']):
        
        agent_loc_ts[agent.iloc[i]['agent']][0] = agent.iloc[i]['lat']
        agent_loc_ts[agent.iloc[i]['agent']][1] = agent.iloc[i]['lon']
        agent_loc_ts[agent.iloc[i]['agent']][2] = date

# Saving a cleaned agent csv with only locations

In [10]:
location_df = pd.DataFrame()

location_df['agent'] = [i for i in agent_loc_ts.keys()]
location_df['lat'] = [i[0] for i in agent_loc_ts.values()]
location_df['lon'] = [i[1] for i in agent_loc_ts.values()]
sale_price = []

for each_agent in location_df['agent']:
    sale_price.append(agent_sale_dict_sorted[each_agent])

location_df['sales'] = sale_price

location_df.to_csv('agent_cleaned.csv', index=False)

In [11]:
location_df.head()

# Getting map plot of all agents (Question 1)

In [12]:
import pandas as pd
import plotly.express as px

fig = px.scatter_mapbox(
    location_df,  # Our DataFrame
    lat = "lat",
    lon = "lon",
    center = {"lat": 24, "lon": 90},  # where map will be centered
    width = 600,  # Width of map
    height = 600,  # Height of map
    hover_data = ["agent"],  # what to display when hovering mouse over coordinate
)

fig.update_layout(mapbox_style="open-street-map") # adding beautiful street layout to map

fig.show()

# Getting map plot of top agents

In [13]:
top_seller_df = pd.DataFrame()
ta = []
lat = []
lon = []
for i, target_agent in enumerate(agent_sale_dict_sorted.keys()):
    if i==100:
        break
    ta.append(target_agent)
    lat.append(agent_loc_ts[target_agent][0])
    lon.append(agent_loc_ts[target_agent][1])


top_seller_df['top_agent'] = ta
top_seller_df['lat'] = lat
top_seller_df['lon'] = lon


fig = px.scatter_mapbox(
    top_seller_df,  # Our DataFrame
    lat = "lat",
    lon = "lon",
    center = {"lat": 24, "lon": 90},  # where map will be centered
    width = 600,  # Width of map
    height = 600,  # Height of map
    hover_data = ["top_agent"],  # what to display when hovering mouse over coordinate
)

fig.update_layout(mapbox_style="open-street-map") # adding beautiful street layout to map

fig.show()

In [14]:
location_df = location_df.sort_values(['lat','lon'])

# Find nearest two agents within 500m

In [15]:
import geopy.distance
import numpy as np

dist=0
dist2=0


agent_arr = location_df['agent'].values
final_list = []
lat_isolate = []
lon_isolate = []

for each_top_seller in tqdm.tqdm(top_seller_df['top_agent']):
    
    idx = np.where(agent_arr == each_top_seller)[0][0]
    lat1 = location_df.iloc[idx]['lat']
    lon1 = location_df.iloc[idx]['lon']
    cord1 = (lat1, lon1)
    if idx == 0:
        lat2 = location_df.iloc[idx+1]['lat']
        lon2 = location_df.iloc[idx+1]['lon']
        cord2 = (lat2, lon2)
        dist = geopy.distance.geodesic(cord1, cord2).m
    elif idx == len(location_df):
        lat2 = location_df.iloc[idx-1]['lat']
        lon2 = location_df.iloc[idx-1]['lon']
        cord2 = (lat2, lon2)
        dist = geopy.distance.geodesic(cord1, cord2).m
    else:
        lat2 = location_df.iloc[idx-1]['lat']
        lon2 = location_df.iloc[idx-1]['lon']
        lat3 = location_df.iloc[idx+1]['lat']
        lon3 = location_df.iloc[idx+1]['lon']
        cord2 = (lat2, lon2)
        cord3 = (lat3, lon3)
        dist = geopy.distance.geodesic(cord1, cord2).km
        dist2 = geopy.distance.geodesic(cord1, cord3).km
    if dist > 0.5 and dist2 > 0.5:
        #print(dist, dist2)
        
        final_list.append(each_top_seller)
        lat_isolate.append(lat1)
        lon_isolate.append(lon1)
        

## final_list length shows the number of isolated agents and final_list contain their agent id

In [16]:
len(final_list)

In [17]:
final_list

# Plot the isolated agents on the map

In [18]:
isolated_df = pd.DataFrame()

isolated_df['agent'] = final_list
isolated_df['lat'] = lat_isolate
isolated_df['lon'] = lon_isolate


fig = px.scatter_mapbox(
    isolated_df,  # Our DataFrame
    lat = "lat",
    lon = "lon",
    center = {"lat": 24, "lon": 90},  # where map will be centered
    width = 600,  # Width of map
    height = 600,  # Height of map
    hover_data = ["agent"],  # what to display when hovering mouse over coordinate
)

fig.update_layout(mapbox_style="open-street-map") # adding beautiful street layout to map

fig.show()

# Ques 3: To find Geo-blocks with retail

* At first we find the map of all agents
* We delete all the outlier locations by sorting the locations
* Then we draw a grid mesh on the map 
* In the grid, x axis and y axis values are calculated using geolocation distance calculator. Using the each grid size of 1.2*0.6 Km^2
* After that we run a grid search on the grid mesh constructed with the geo block size
* Each grid find atleast 1 retail is added lastly to find total geo-blocks with retail

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

# Showing the geoblocks with grid on map

In [ ]:
df = pd.read_csv("../input/agent-modified/agent_cleaned.csv")

#excluding the outliers outside Bangladesh
df = df.loc[df['lon'] >= 88]
df = df.loc[df['lon'] < 92.5]
df = df.loc[df['lat'] >= 21]
df = df.loc[df['lat'] < 27]

#df.plot()  # plots all columns against index

#plt.style.use('fivethirtyeight')

maximum_lon = df['lon'].max()
minimum_lon = df['lon'].min()

maximum_lat = df['lat'].max()
minimum_lat = df['lat'].min()

#using calculator: https://www.meridianoutpost.com/resources/etools/calculators/calculator-latitude-longitude-distance.php?
y_ticks = np.arange(minimum_lat, maximum_lat, 0.005)    #latitudes
x_ticks = np.arange(minimum_lon, maximum_lon, 0.012)  #longitudes

fig, ax = plt.subplots()
ax.scatter(df['lon'], df['lat'])
ax.set_yticks(y_ticks, minor=False)
ax.yaxis.grid(True, which='major')

ax.set_xticks(x_ticks, minor=False)
ax.xaxis.grid(True, which='major')
ax.set_yticklabels([])
ax.set_xticklabels([])

set_size(18,18)
plt.show()

# Finding total number of geo-blocks

In [ ]:
from tqdm import tqdm
def number_of_geoblocks(long, latit, df):
    total_lon = len(long)
    total_lat = len(latit)
    
    geo_block_retail = 0
    
    print(f'Total geo-block is {(total_lon+1)*(total_lat+1)}')
    
    for i in tqdm(range(len(x_ticks)-1)):
        a = (df['lon'] < x_ticks[i+1]) & (df['lon'] >= x_ticks[i])
        
        for j in (range(len(y_ticks)-1)):
            b = (df['lat'] < y_ticks[j+1]) & (df['lat'] >= y_ticks[j])  
            result = a & b

            if(result.any()):
                geo_block_retail = geo_block_retail+1
                
        #print(f'Total retail geoblock is {geo_block_retail} and total')    
    return geo_block_retail


In [ ]:
# print total number of geo block with retail

print(number_of_geoblocks(x_ticks,y_ticks,df))

# Qus 4: Targeted 10 Retailer deployment geo blocks
* We will first sort the dataframe based on total sales of each agent in decending order
* Then will start checking from top selling agent that if a 3x3 neighbourhood area is free of any other retailer
* If whole 3x3 neighborhood is empty of any retailer then we again check for that geohash's 3x3 neighborhood
* If neighborhood of empty geohash is also empty then we can finally target that geo block

In [ ]:
##partho if you can provide me the df with lon,lat,sales from above then remove this cell



## data loading with total sales
df_sale = pd.read_csv("./agent_cleaned.csv")
df_sale.head()


final = df_sale


# Get the sorted based on total sales. We will start with top seller

In [ ]:
# sort based on sales
sorted_df = df.sort_values(by=['sales'], ascending=False)
sorted_df.head()

# Necessary helper functions

In [ ]:
x_ticks = list(x_ticks)
y_ticks = list(y_ticks)

def visualize_in_geoblock(co_ord, df):   
    ## it will visualize the targeted coordinates among the map
    topten = pd.DataFrame(co_ord,columns =['lon','lat'])
    
    fig, ax = plt.subplots()
    ax.scatter(df['lon'], df['lat'],c=df['sales'], s=15, cmap='viridis')   #cmap = 'RdBu',
    ax.set_yticks(y_ticks, minor=False)
    ax.yaxis.grid(True, which='major')
    ax.set_xticks(x_ticks, minor=False)
    ax.xaxis.grid(True, which='major')
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    
    ax.scatter(topten['lon'], topten['lat'], s=80,edgecolors='red', color = 'yellow')   #cmap = 'RdBu',
    set_size(18,18)
    plt.show()


def find_neighbour(lon, lat, df):
    ## this will find the maximum salers grid then search for neighbourhood 3*3 kernel if there is not any nearest retailer. 
    ## if there is not retailer around them then we can put one in that empty geo-block
    ## input: longitude, latitude and full df
    ## output: list of desired possible positions of targeted geo-blocks
    
    global x_ticks,y_ticks
    
    index_lon = min(range(len(x_ticks)), key=lambda i: abs(x_ticks[i]-lon))   #find the nearest grid line
    index_lat = min(range(len(y_ticks)), key=lambda i: abs(y_ticks[i]-lat))
 
    ###############
    ### a  b  c ###
    ### d  *  e ###
    ### f  g  h ###
    ###############
    
    #for i in (range(len(x_ticks)-1)):
    ax = (df['lon'] >= x_ticks[index_lon-1]) & (df['lon'] < x_ticks[index_lon])
    ay = (df['lat'] >= y_ticks[index_lat+1]) & (df['lat'] < y_ticks[index_lat])
    a = ax & ay
    
    bx = (df['lon'] >= x_ticks[index_lon]) & (df['lon'] < x_ticks[index_lon+1])
    by = ay #(df['lat'] >= y_ticks[index_lat+1]) & (df['lat'] < y_ticks[index_lat])
    b = bx & by
    
    cx = (df['lon'] >= x_ticks[index_lon+1]) & (df['lon'] < x_ticks[index_lon+2])
    cy = ay  #(df['lat'] >= y_ticks[index_lat+1]) & (df['lat'] < y_ticks[index_lat])
    c = cx & cy
    
    dx = ax
    dy = (df['lat'] >= y_ticks[index_lat]) & (df['lat'] < y_ticks[index_lat-1])
    d = dx & dy
    
    ex = cx
    ey = dy
    e = ex & ey
    
    fx = ax
    fy = (df['lat'] >= y_ticks[index_lat-1]) & (df['lat'] < y_ticks[index_lat-2])
    f = fx & fy
    
    gx = bx
    gy = fy
    g = gx & gy
    
    hx = cx
    hy = fy
    h = hx & hy
    
    empty_found = (~(a.any() | b.any() | c.any() |d.any() |e.any() |f.any() |g.any() |h.any()))
    
    return empty_found,a,b,c,d,e,f,g,h,index_lon,index_lat


def selecting_neighbor(empt,a,b,c,d,e,f,g,h,df, index_lon, index_lat):
    global x_ticks,y_ticks
    
    if(empt):
        #for each geohash:
        if(~a.any()):
            y,_,_,_,_,_,_,_,_,_,_ = find_neighbour(x_ticks[index_lon-1],y_ticks[index_lat+1], df)  #find neighborhood 3*3 for this empty geohash
            if(y):
                co_ordinate = [x_ticks[index_lon-1],y_ticks[index_lat+1]]
        if(~b.any()):
            y,_,_,_,_,_,_,_,_,_,_ = find_neighbour(x_ticks[index_lon],y_ticks[index_lat+1], df)
            if(y):
                co_ordinate = [x_ticks[index_lon],y_ticks[index_lat+1]]
        if(~c.any()):
            y,_,_,_,_,_,_,_,_,_,_ = find_neighbour(x_ticks[index_lon+1],y_ticks[index_lat+1], df)
            if(y):
                co_ordinate = [x_ticks[index_lon+1],y_ticks[index_lat+1]]
        if(~d.any()):
            y,_,_,_,_,_,_,_,_,_,_ = find_neighbour(x_ticks[index_lon-1],y_ticks[index_lat], df)
            if(y):
                co_ordinate = [x_ticks[index_lon-1],y_ticks[index_lat]]
        if(~e.any()):
            y,_,_,_,_,_,_,_,_,_,_ = find_neighbour(x_ticks[index_lon+1],y_ticks[index_lat], df)
            if(y):
                co_ordinate = [x_ticks[index_lon+1],y_ticks[index_lat]]
        if(~f.any()):
            y,_,_,_,_,_,_,_,_,_,_ = find_neighbour(x_ticks[index_lon-1],y_ticks[index_lat-1], df)
            if(y):
                co_ordinate = [x_ticks[index_lon-1],y_ticks[index_lat-1]]
        if(~g.any()):
            y,_,_,_,_,_,_,_,_,_,_ = find_neighbour(x_ticks[index_lon],y_ticks[index_lat-1], df)
            if(y):
                co_ordinate = [x_ticks[index_lon],y_ticks[index_lat-1]]
        if(~h.any()):
            y,_,_,_,_,_,_,_,_,_,_ = find_neighbour(x_ticks[index_lon+1],y_ticks[index_lat-1], df)
            if(y):
                co_ordinate = [x_ticks[index_lon+1],y_ticks[index_lat-1]]
        
        return co_ordinate
    
    else:
        return None
    


# Finding the coordinates of targeted 10 geo blocks

In [ ]:
i = 0
coordinates = []
while(i<10):
    element = sorted_df.iloc[i]
    longi = element['lon']
    latit = element['lat']
    y,a,b,c,d,e,f,g,h,ind_lon, ind_lat = find_neighbour(longi, latit, df)
    cc = selecting_neighbor(y,a,b,c,d,e,f,g,h,df,ind_lon,ind_lat)
    if(cc!= None):
        coordinates.append(cc)
        i += 1

In [ ]:
#to visualize all 10 points in geo block map
visualize_in_geoblock(coordinates, df)

# Qus 11: Next big billboard placement
* We will find the highest number of agent from the geo blocks in task 3. Because most number of agent in a geo block denotes mostly used area for business.
* We will register the geographical position of that geo block containing highest number of agent

In [ ]:
from tqdm import tqdm

def number_of_agents_in_geoblocks(long, latit, df):
    total_lon = len(long)
    total_lat = len(latit)
    total_agent_in_geoblock = 0
    top_agents = 0
    top_agents_pos = []    #will save highest number of agents in a geo block
    
    print(f'Total geo-block is {(total_lon+1)*(total_lat+1)}')
    
    for i in tqdm(range(len(x_ticks)-1)):
        a = (df['lon'] < x_ticks[i+1]) & (df['lon'] >= x_ticks[i])
        
        for j in (range(len(y_ticks)-1)):
            b = (df['lat'] < y_ticks[j+1]) & (df['lat'] >= y_ticks[j])  
            result = a & b
            total_agent_in_geoblock = sum(result)
            if(total_agent_in_geoblock > top_agents):    # we will take the highest number of agents from the grid search in all geo blocks
                top_agents = total_agent_in_geoblock
                top_agents_pos = [x_ticks[i], y_ticks[j]]

                    
    return top_agents, top_agents_pos


In [ ]:
agent, pos = number_of_agents_in_geoblocks(x_ticks,y_ticks,df)

In [ ]:
#to visualize the mostly important point in geo block map
visualize_in_geoblock(pos, df)

# Qus 5 & 9: Peak hour estimation

# Our intuition is that the customer who did maximum transaction traveled most. Based on this intuition, we determined the top 20 customer with highest number transaction.

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#download the data from kaggle
!kaggle datasets download -d parthoghosh/synaptican-dataset
!unzip \*.zip  && rm *.zip

In [ ]:
import os
import pandas as pd
data_list = os.listdir('/content/transaction') #listing all the csv file in the transaction directory
dataset_list = sorted([os.path.join('transaction',i) for i in data_list]) #sorting alphanumerically (optional)

### This cell creates product-price dictionary pair where products are used as keys and prices are used as values

In [ ]:
product_df = pd.read_csv('product.csv') #loading product.csv as a pandas dataframe
key_list = product_df['product'].values # listing all the product name
value_list = product_df['price'].values #listing all the price
zip_iterator = zip(key_list, value_list) # zipping the product-price pair
price_dictionary = dict(zip_iterator) # converting them into a dictionary

### This cell calculates the total sales and total transaction by hour for every csv file and append them to two separate list [sales list and transactions_list] 

In [ ]:
sales_list = [] #declaring sales list 
transaction_list = [] # declaring transaction list
for dataset in dataset_list:  
  df= pd.read_csv(dataset) #converting csv to dataframe
  df['Hours'] = df['ts'].apply(lambda x : x.split(' ')[1].split(':')[0]) #getting the hour value from time stamp string
  df = df.drop(['agent','customer','dt','ts'], axis =1) #droping unnecessary columns
  df['Price'] = df['product'].apply(lambda x : price_dictionary[x]) #creating the price colmun according to the product
  sales_list.append(df.groupby('Hours').sum()) #appends in to the sales list a dataframe containing total sales by hour
  transaction_list.append(df['Hours'].value_counts()) # appends in to the transaction list a series object containing total number of product sold by hour

### We sum up all the hourly sales .The peak hour for sale is 19.00h as it can be seen after sorting in the output of the following cell

In [ ]:

total_sales = sales_list[0]
for i in sales_list[1:]:
  total_sales += i
total_sales = total_sales.sort_values(by='Price',ascending=False)
total_sales

### We sum up all the hourly transaction .The peak hour for number of transaction is 19.00h as it can be seen in the output of the following cell

In [ ]:
total_transaction = transaction_list[0]
for i in transaction_list[1:]:
  total_transaction += i
total_transaction

In [ ]:
from re import I
customer_list = []
for i in dataset_list :
  print(i)
  data = pd.read_csv(i)
  for i in data['customer'].values :
    customer_list.append(i)


In [ ]:
new_list = []
data = pd.read_csv(dataset_list[0])
grouped_customer = data.groupby("customer")
grouped_agent = grouped_customer["agent"].apply(list)
grouped_agent = grouped_agent.reset_index()
grouped_agent

In [ ]:
customer_list = []
for i in dataset_list :
  print(i)
  data = pd.read_csv(i)
  for i in data['customer'].values :
    customer_list.append(i)

In [ ]:
custom_data = pd.DataFrame()

custom_data['customer'] = customer_list

custom_data.value_counts()

# Finding the Retail Agents Serving highest number of travellers (Question No 10)

* First read the transaction csv
* Then find out the top 5 retail who has the most transactions
* Then for those agent find out the unique customer they have served
* Doing it for all the 20 transaction files and storing it in a dictionary where key is agent and value is the array of customer they have served
* finally sorting that dictionary for value to find the best server retailer

In [ ]:
import pandas as pd
import numpy as np
import tqdm
agent = pd.read_csv('../input/synaptican-dataset/agent.csv')
product = pd.read_csv('../input/synaptican-dataset/product.csv')
#transaction = pd.read_csv('../input/synaptican-dataset/transaction/part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv')

In [ ]:

agent_serve_dict = {}

for each_agent in agent['agent'].unique():
    agent_serve_dict[each_agent] = np.array([])

In [ ]:
for i in range(20):
    if i<10:
        i = f'0{i}'
    
    transaction = pd.read_csv(f'../input/synaptican-dataset/transaction/part-000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv')
    
    df_high_count = transaction['agent'].value_counts().reset_index().head()
    print("\n\n\n")
    print(df_high_count)
    print("\n\n\n")

    for each_agent in tqdm.tqdm(df_high_count['index']):
        df_temp = transaction.loc[transaction['agent'] == each_agent]
        arr = df_temp['customer'].unique()
        agent_serve_dict[each_agent] = np.append(agent_serve_dict[each_agent], arr)


# Printing out the best retailer (top 5)

In [ ]:
for each_agent in agent['agent'].unique():
    if agent_serve_dict[each_agent].shape[0] != 0 and not isinstance(agent_serve_dict[each_agent], int):
        agent_serve_dict[each_agent] = len(np.unique(agent_serve_dict[each_agent]))#len(set(agent_serve_dict[each_agent]))
    else:
        agent_serve_dict[each_agent] = 0

agent_serve_dict = dict(sorted(agent_serve_dict.items(), key=lambda item: item[1], reverse=True))

for i,(k,v) in enumerate(agent_serve_dict.items()):
    print(k)
    print(v)
    if i==9:
        break

# Question 7 (A)

# To forecast daily sales for the next week we did -

* First we generate the total sale in each day of the year
* The total dataset contains data of 175 days and we calculated the total sale in each and every day
* Then we use the State-of-the-art forecasting model *Prophet* to forecast the sales for the furture days(next week)
* We also use other visualization to demonstrate the trends of sales (like days and months sales)

In [ ]:
import pandas as pd

agent = pd.read_csv('../input/synaptican-dataset/agent.csv')
product = pd.read_csv('../input/synaptican-dataset/product.csv')
#transaction = pd.read_csv('../input/synaptican-dataset/transaction/part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv')

In [ ]:
import tqdm
import numpy as np
dt = np.array([])
price = np.array([])
train_df = pd.DataFrame()
for i in tqdm.tqdm(range(20)):
    if i<10:
        i = f'0{i}'
    
    transaction = pd.read_csv(f'../input/synaptican-dataset/transaction/part-000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv')
    df = pd.merge(transaction[['dt', 'product']], product[['product', 'price']], on=['product'])
    df = df.groupby(['dt']).sum().reset_index()
    dt = np.append(dt, df['dt'].values)
    price = np.append(price, df['price'].values)
    
    


In [ ]:
train_df['dt'] = dt
train_df['price']=price

In [ ]:
train_df = train_df.groupby(['dt']).sum().reset_index()

In [ ]:
train_df.head()

In [ ]:


import warnings; 
warnings.simplefilter('ignore')



# Installing the necessary packages

In [ ]:
!pip install pystan
!pip install fbprophet

In [ ]:
import pandas as pd
from fbprophet import Prophet

### Making the dataframe ready to training

In [ ]:

train_df['Year'] = train_df['dt'].apply(lambda x: str(x)[:4])
train_df['Month'] = train_df['dt'].apply(lambda x: str(x)[4:6])
train_df['Day'] = train_df['dt'].apply(lambda x: str(x)[6:-2])

train_df['ds'] = pd.DatetimeIndex(train_df['Year']+'-'+train_df['Month']+'-'+train_df['Day'])


In [ ]:
train_df.head()

In [ ]:
train_df.drop(['dt','Year', 'Month', 'Day'], axis=1, inplace=True)
train_df.columns = ['y', 'ds']

In [ ]:
m = Prophet(interval_width=0.95, daily_seasonality=True)
model = m.fit(train_df)

In [ ]:


future = m.make_future_dataframe(periods=7,freq='D')
forecast = m.predict(future)
forecast.head()


In [ ]:
forecast.tail()

### Plotting forecasts

In [ ]:

plot1 = m.plot(forecast)

In [ ]:
plot1.savefig('forecast1.png')

In [ ]:
plt2 = m.plot_components(forecast)

In [ ]:
plt2.savefig('forecast2.png')

# Question (7 B)

# Next day’s sales of each retail
* There are more than 100k unique retail seller
* so to forecast their sales we used the mathematical approach
* we created a data frame and a csv file containing the agen id and their maximum sale price and minimum sale price and mean sale price
* mean sale price will be that agent's next day's sale

#### NB: If you face ram issue then restrat and run , each question section is a unique section not dependant on one other

In [ ]:
import pandas as pd

agent = pd.read_csv('../input/synaptican-dataset/agent.csv')
product = pd.read_csv('../input/synaptican-dataset/product.csv')
#transaction = pd.read_csv('../input/synaptican-dataset/transaction/part-00001-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv')

In [ ]:
agent_sales_dict = {}

for each_agent in agent['agent'].unique():
    agent_sales_dict[each_agent] = [0,0,0]


In [ ]:
import tqdm
import numpy as np
for i in range(20):
    if i<10:
        i = f'0{i}'
    
    transaction = pd.read_csv(f'../input/synaptican-dataset/transaction/part-000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv')
    
    df = pd.merge(transaction[['agent','dt', 'product']], product[['product', 'price']], on=['product'])
    df_max = df.groupby(['agent']).max().reset_index()
    df_min = df.groupby(['agent']).min().reset_index()
    df_mean = df.groupby(['agent']).mean().reset_index()
    
    for i in tqdm.tqdm(range(len(df_mean))):
        l1 = agent_sales_dict[df_max.iloc[i]['agent']]
        l2 = [df_max.iloc[i]['price'], df_min.iloc[i]['price'], df_mean.iloc[i]['price']]
        agent_sales_dict[df_max.iloc[i]['agent']] = [sum(x) for x in zip(l1, l2)]


In [ ]:
all_agent = list(agent_sales_dict.keys())
all_mean = [i[2] for i in agent_sales_dict.values()]
all_max = [i[0] for i in agent_sales_dict.values()]
all_min = [i[1] for i in agent_sales_dict.values()]

### Generating the csv and the data frame

In [ ]:
agent_sale_forecasting = pd.DataFrame()
agent_sale_forecasting['agent'] = all_agent
agent_sale_forecasting['max sale'] = all_max
agent_sale_forecasting['min sale'] = all_min
agent_sale_forecasting['mean sale'] = all_mean

agent_sale_forecasting.to_csv('./agent_sale_forecasting.csv', index=False)

In [ ]:
agent_sale_forecasting.head()

In [ ]:
agent_sale_forecasting

# Question 8 Area Categorization

Here to solve the challenge, our intuition is that residential areas, transit hubs, and corporate areas show different transaction frequencies based on the time hour. On the other hand, tourist spots' traction varies with the month. Moreover, we have seen that transaction history with respect to time is almost similar for all 20 transaction csv. (The plot is shown in the presentation). That's why we demonstrated our algorithm for one transaction csv.

First, we extracted the Tourist spots based on the moth analysis. Then we extracted the remaining places. Before doing that, we first clean the agent csv. (remove the outlrs and nan value)

In [ ]:
agent=pd.read_csv("agent_cleaned.csv")
agent=agent.loc[(agent['lat'] <= 26) & (21 <= agent['lat']) & (88 <= agent['lon']) & (agent['lon'] <= 92.5)]

# Tourist spots

In [ ]:
Data=pd.read_csv('data/part-00001-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv') # Read one transaction csv file


 Selct the common agents in agent and Data csv

In [ ]:
agent_org=set(agent['agent'].unique())
agent_tran=set(Data['agent'].unique())

agent_org_tran=agent_org.intersection(agent_tran)

agent_org_tran=list(agent_org_tran)

data=Data.loc[Data['agent'].isin(agent_org_tran)] # After agent filtering 

In [ ]:
data=data.drop(['customer','product','dt'],axis=1) # drop customer, product,dt

### From time stand, extract only month

In [ ]:
data['month']=[int(x.split(' ')[0].split('-')[1]) for x in data['ts']]
data=data.drop(['ts'],axis=1)

In [ ]:
data_agent = data.groupby("agent")
data_agent_month = data_agent["month"].apply(list)
data_agent_month = data_agent_month.reset_index()
data_agent_month

### Here we calclulated the variance of the sell month. Intuitively, the tourist spots shuld have highest varinace 

In [ ]:
data_agent_month['var']=[np.var(x) for x in data_agent_month['month']]

#### Choose the agents which have highest varinace.

In [ ]:
data_tourist=data_agent_month[data_agent_month['var']==max(data_agent_month['var'])]

In [ ]:
data_tourist=data_tourist.drop(['month','var'],axis=1)

In [ ]:
agent_tourist=agent.loc[agent['agent'].isin(data_tourist['agent'].values)].reset_index() # Agent tourist

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(
    agent_tourist,  # your DataFrame which have lattitude and longitude
    lat = "lat",  # lattitude column name	
    lon = "lon",  # longitude column name
    center = {"lat": 24, "lon": 90},  # where map will be centered (do not need to change)
    width = 600,  # Width of map
    height = 600,  # Height of map
    hover_data = ["agent"],  # what to display when hovering mouse over coordinate
    color='class',
)

fig.update_layout(mapbox_style="open-street-map") # adding beautiful street layout to map

fig.show()

# Residential areas, Transit hubs, and Corporate areas

In [ ]:
agent_org=set(agent['agent'].unique())
agent_tran=set(Data['agent'].unique())
agent_tourist=set(agent_tourist['tourist'].values)

agent_tran=agent_tran-agent_tourist # Remove the tourist agent

agent_org_tran=agent_org.intersection(agent_tran)

agent_org_tran=list(agent_org_tran)

data=Data.loc[Data['agent'].isin(agent_org_tran)] # After agent filtering 

In [ ]:
data=data.drop(['customer','product','dt'],axis=1) # drop customer, product,dt

#### For each agent, group all the transition hour time

In [ ]:
data['hour']=[int(x.split(' ')[1].split(':')[0]) for x in data['ts']]

In [ ]:
data_hour=data.groupby("agent")
data_agent = data.groupby("agent")
data_agent_hour = data_agent["hour"].apply(list)
data_agent_hour = data_agent_hour.reset_index()
data_agent_hour

Based on the transaction time, we make 3 classes

class=0 (Transit hubs)
class=1 (Corporate areas)
class=2 (Residential areas)

In [ ]:
data_agent_hour['class'] = pd.Series(np.random.randn(len(data_agent_hour)), index=data_agent_hour.index)

In [ ]:
for ind,hl in enumerate (data_agent_hour['hour']):
    
    x=0
    y=0
    z=0
    
    for h in hl:
        if (0<h<9):
            x=x+1
        elif (9<h<18):
            y=y+1
        else:
            z=z+1
            
    if (x>y & x>z):
        data_agent_hour['class'][ind]=0
        
    elif (y>x & y>z):
        data_agent_hour['class'][ind]=1
    else:
        data_agent_hour['class'][ind]=2

Make 3 datframes for 3 regions. 

agent_tran (Transit hubs)
agent_cor (Corporate areas)
agent_cor (Residential areas)

In [ ]:
agent_trans=agent.loc[agent['agent'].isin(data_agent_hour[data_agent_hour['class']==0]['agent'].values)].reset_index() 
agent_cor=agent.loc[agent['agent'].isin(data_agent_hour[data_agent_hour['class']==1]['agent'].values)].reset_index() 
agent_res=agent.loc[agent['agent'].isin(data_agent_hour[data_agent_hour['class']==2]['agent'].values)].reset_index() 


Finally, we plot the agents location in map.

For clarity, we ploted agent_res in seperate figure. To see the distincst location we ploted agent_trans and agent_cor in same figure

In [ ]:
agent_trans_cop=pd.concat([agent_corp, agent_trans])
agent_trans_cop # merge agent_corp, agent_trans

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(
    agent_trans_cop,  # your DataFrame which have lattitude and longitude
    lat = "lat",  # lattitude column name	
    lon = "lon",  # longitude column name
    center = {"lat": 24, "lon": 90},  # where map will be centered (do not need to change)
    width = 600,  # Width of map
    height = 600,  # Height of map
    hover_data = ["agent"],  # what to display when hovering mouse over coordinate
    color='class',
)

fig.update_layout(mapbox_style="open-street-map") # adding beautiful street layout to map

fig.show()

 Plot resedential areas

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(
    agent_res,  # your DataFrame which have lattitude and longitude
    lat = "lat",  # lattitude column name	
    lon = "lon",  # longitude column name
    center = {"lat": 24, "lon": 90},  # where map will be centered (do not need to change)
    width = 600,  # Width of map
    height = 600,  # Height of map
    hover_data = ["agent"],  # what to display when hovering mouse over coordinate
    
)

fig.update_layout(mapbox_style="open-street-map") # adding beautiful street layout to map

fig.show()

# Question 6
The following portion of the notebook calculates total sales by hour per moth.

Almost all the months shows similar kind of sale distribution over time .The month of january(month1) hase peak sales rate at 18h and for rest of the months peak sale is at 19h.This is because days are smaller in the month of january. From the graph we can see that high sale rate happens from 17th-20th hour when most of the offices are over.

In [ ]:
import os
import pandas as pd
data_list = os.listdir('/content/transaction') #listing all the csv file in the transaction directory
dataset_list = sorted([os.path.join('transaction',i) for i in data_list]) #sorting alphanumerically (optional)

### This cell creates product-price dictionary pair where products are used as keys and prices are used as values

In [ ]:
product_df = pd.read_csv('product.csv') #loading product.csv as a pandas dataframe
key_list = product_df['product'].values # listing all the product name
value_list = product_df['price'].values #listing all the price
zip_iterator = zip(key_list, value_list) # zipping the product-price pair
price_dictionary = dict(zip_iterator) # converting them into a dictionary

### This cell calculates the total sales and total transaction by hour for every csv file and append them to two separate list [sales list and transactions_list] 

In [ ]:
sales_list = [] #declaring sales list 
transaction_list = [] # declaring transaction list
months_sale = []
for dataset in dataset_list: 
  print(dataset) 
  df= pd.read_csv(dataset) #converting csv to dataframe
  df['Hours'] = df['ts'].apply(lambda x : x.split(' ')[1].split(':')[0]) #getting the hour value from time stamp string
  df['Months'] = df['ts'].apply(lambda x : x.split('-')[1])
  df = df.drop(['agent','customer','dt','ts'], axis =1) #droping unnecessary columns
  df['Price'] = df['product'].apply(lambda x : price_dictionary[x]) #creating the price colmun according to the product
  sales_list.append(df.groupby('Hours').sum()) #appends in to the sales list a dataframe containing total sales by hour
  transaction_list.append(df['Hours'].value_counts()) # appends in to the transaction list a series object containing total number of product sold by hour
  months_sale.append(df.groupby(['Months','Hours']).sum())

In [ ]:
total_sales = months_sale[0]
for i in months_sale[1:]:
  total_sales += i
#total_sales = total_sales.sort_values(by='Price',ascending=False)
# total_sales

In [ ]:
month1 = total_sales[total_sales['Months']=='01']
month2 =  total_sales[total_sales['Months']=='02']
month3 =  total_sales[total_sales['Months']=='03']
month4 =  total_sales[total_sales['Months']=='04']
month5 =  total_sales[total_sales['Months']=='05']
month6 =  total_sales[total_sales['Months']=='06']

In [ ]:
month1.plot.bar(x='Hours', y='Price', rot=0)


In [ ]:
month2.plot.bar(x='Hours', y='Price', rot=0)

In [ ]:
month3.plot.bar(x='Hours', y='Price', rot=0)

In [ ]:
month4.plot.bar(x='Hours', y='Price', rot=0)

In [ ]:
month5.plot.bar(x='Hours', y='Price', rot=0)

In [ ]:
month6.plot.bar(x='Hours', y='Price', rot=0)

### Question 9 : IN the following cells we calculate tatol number of interaction happend between a customer and different retailer 
Based on these interaction we can take a guess which customers travels the most.The higher the number of interaction , the more active customer

In [ ]:
customer_list = []
for i in dataset_list :
  print(i)
  data = pd.read_csv(i)
  for i in data['customer'].values :
    customer_list.append(i)



In [ ]:
custom_data = pd.DataFrame()

custom_data['customer'] = customer_list


In [ ]:
customer_interaction_count = custom_data.value_counts()

In [ ]:
customer_interaction_count.head(20) #showing the top 20 active customers